                                                                        Лабораторна робота 2                                                                                     

1) Створити env в якому будуть встановлені всі необхідні бібліотеки та
налаштування для данної лабораторної роботи

<a href="https://imgbb.com/"><img src="https://i.ibb.co/GvvFz9X/image.png" alt="image" border="0"></a>

<a href="https://imgbb.com/"><img src="https://i.ibb.co/1QzxQsc/p6v-Xjs-P-SLOPgqu9-ZNXou-A.png" alt="p6v-Xjs-P-SLOPgqu9-ZNXou-A" border="0"></a>


In [4]:
import pandas as pd

2) Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу. Ця процедура
має бути автоматизована, параметром процедури має бути індекс (номер) області. При зберіганні файлу до його імені потрібно додати
дату та час завантаження;

In [5]:
import urllib.request
from datetime import datetime
import re
import os 

dt = datetime.now()
dt_string = dt.strftime("%d_%m_%Y_%H:%M")

def vhi_download(id):
        if not any (file for file in os.listdir('vhi') if file.split('_',2)[0] == f"vhi{id}"):
            urllib.request.urlretrieve(f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={id}&year1=1981&year2=2024&type=Mean", f"vhi/vhi{id}_{dt_string}.csv")
    
for i in range (1,28):
   vhi_download(i)   

3) Зчитати завантажені текстові файли у фрейм (детальніше про роботу із фреймами буде розказано у подальших лабораторних роботах).Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли;

In [6]:
import os 

def csv_to_dateframe(directory):
    dfs = {} 
    for file in os.listdir(directory):
        key = file.split('_', 2)[0]
        dfs[key] = pd.read_csv(f'{directory}/{file}', index_col=False, header=1)
        dfs[key].columns = dfs[key].columns.str.strip()
        dfs[key].columns = dfs[key].columns.str.lower()
        dfs[key].rename(columns={'vhi<br>': 'vhi'}, inplace=True)
    return dfs

dfs = csv_to_dateframe('vhi')
print(dfs['vhi20'].head())

            year  week    smn     smt    vci    tci    vhi
0  <tt><pre>1982   1.0  0.054  263.39  17.56  67.52  42.54
1           1982   2.0  0.056  264.47  17.35  64.82  41.09
2           1982   3.0  0.059  265.96  18.50  60.91  39.70
3           1982   4.0  0.061  268.24  19.39  54.00  36.69
4           1982   5.0  0.066  269.84  22.14  49.24  35.69


4) Реалізувати процедуру, яка змінить індекси областей, які використані на порталі NOAA на наступні (виключно старі індекси на нові):
<a href="https://imgbb.com/"><img src="https://i.ibb.co/dbqcq98/image.png" alt="image" border="0"></a>

In [7]:
new_ids = {'Mykolayiv': '13',
 'Kiev': '9',
 'Zhytomyr': '5',
 'Kirovohrad': '10',
 "Luhans'k": '11',
 'Sumy': '17',
 'Rivne': '16',
 'Crimea': '25',
 "Dnipropetrovs'k": '3',
 'Volyn': '2',
 "Khmel'nyts'kyy": '21',
 'Cherkasy': '22',
 'Transcarpathia': '6',
 "Ivano-Frankivs'k": '8',
 "Donets'k": '4',
 'Kherson': '20',
 'Odessa': '14',
 'Kharkiv': '19',
 "L'viv": '12',
 'Zaporizhzhya': '7',
 'Chernihiv': '24',
 'Vinnytsya': '1',
 'Chernivtsi': '23',
 "Ternopil'": '18',
 'Poltava': '15',
 'Kiev City' : '26',
 "Sevastopol'" : '27'
}

#os.mkdir('vhi_modified')

def change_ids(dfs, ids, directory='vhi'):
    for file in os.listdir(directory):
        file_path = f'{directory}/{file}'
        with open(file_path, 'r') as f:
            first_line = f.readline().strip()
            province = first_line.split(",", 3)[0].split(':', 2)[1].strip()
            current_id = file.split('_', 2)[0].split("vhi",1)[1]
            desired_id = ids[f'{province}']
            if current_id != desired_id and not any (file for file in os.listdir('vhi_modified') if file.split('_',2)[0] == f"vhi{current_id}"):
                os.rename(file_path, f'vhi_modified/{file.replace(f"vhi{current_id}", f"vhi{desired_id}")}')
    return csv_to_dateframe('vhi_modified')


dfs_mod = change_ids(dfs, new_ids)

print(dfs_mod['vhi1'].head())

            year  week    smn     smt    vci    tci    vhi
0  <tt><pre>1982   1.0  0.068  263.59  63.47  28.34  45.90
1           1982   2.0  0.074  265.78  67.62  23.05  45.34
2           1982   3.0  0.076  267.19  69.37  20.40  44.88
3           1982   4.0  0.075  268.57  65.26  17.93  41.60
4           1982   5.0  0.072  269.24  58.58  20.00  39.29


5) Реалізувати процедури для формування вибірок наступного виду
(включаючи елементи аналізу):

        o Ряд VHI для області за вказаний рік;

In [8]:
def vhi_year(df, year):
    return df.loc[(df["year"] == year) & (df['vhi'] != -1), "vhi"]

vhi1_1999 = vhi_year(dfs_mod["vhi22"], '2024')

print(vhi1_1999.head())

2184    41.91
2185    44.19
2186    48.12
2187    49.32
2188    46.79
Name: vhi, dtype: float64


        o Пошук екстремумів (min та max) для вказаних областей та років;

In [9]:
def vhi_min_year(df, year):
    return vhi_year(df,year).min()

def vhi_max_year(df, year):
    return vhi_year(df,year).max()

def get_province(id):
    return {i for i in new_ids if new_ids[i] == id}.pop()

id = "12"
df = dfs_mod[f"vhi{id}"]
province =  {i for i in new_ids if new_ids[i] == id}.pop()
year = '2012'

print(f"Minimum VHI for {province} in {year}: {vhi_min_year(df, year)}")
print(f"Maximum VHI for {province} in {year}: {vhi_max_year(df, year)}")


Minimum VHI for L'viv in 2012: 41.64
Maximum VHI for L'viv in 2012: 65.95


        o Ряд VHI за вказаний діапазон років для вказаних областей;

In [11]:
for df in dfs_mod.keys():
    id = df.split("vhi",2)[1]
    dfs_mod[f'vhi{id}']['year'] = dfs_mod[f'vhi{id}']['year'].astype(str).str.replace('<tt><pre>', '').str.replace('</pre></tt>', '')
    #dfs_mod[f'vhi{id}'] = dfs_mod[f'vhi{id}'].drop(2236)

def vhi_range(ids, start, end):
    vhi_range = {}
    for id in ids:
        df = dfs_mod[f'vhi{id}']
        df["year"] = df["year"].astype(int)
        vhi_range[f'{id}'] = df.loc[ (df["year"] >= start) & (df["year"] <= end), "vhi"]
    return vhi_range

ids = ['10', '14', '6', '4']
start = 2005
end = 2012

vhi_r = vhi_range(ids, start, end)
print(f'VHI for {get_province(ids[3])} from {start} to {end}: \n{vhi_r[ids[3]]}')

VHI for Donets'k from 2005 to 2012: 
1196    69.37
1197    74.87
1198    76.84
1199    75.70
1200    75.60
        ...  
1607    43.65
1608    40.39
1609    38.69
1610    40.15
1611    43.84
Name: vhi, Length: 416, dtype: float64


        o Для всього набору даних виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей по Україні (20% областей - 5 областей з 25). Повернути роки, назви областей з екстремальними посухами та значення VHI;

In [12]:
def years_drought(dfs):
    years_drought = {}
    for df in dfs.items():
        id = df[0].split("vhi",2)[1]
        series_dr= df[1][(df[1].vhi < 15) & (df[1].vhi != -1)]
        years_dr = series_dr['year'].unique()
        years_drought[f'{id}'] = years_dr 
    
    summary = []

    for k in years_drought.items():
        summary.extend(k[1])

    massive_drought = {i for i in summary if summary.count(i)>5}
    vhis = {}
    for k in years_drought.items():
        for i in massive_drought:
            if i in k[1]:
                id = k[0]
                province = get_province(id)
                df = dfs[f'vhi{id}']
                vhi = (df[(df.vhi < 15) & (df.vhi != -1)][['year','vhi']])
                vhis[f'{province}'] = vhi
    return massive_drought, vhis

print (years_drought(dfs_mod))

({'2000'}, {'Kiev':      year    vhi
979  2000  12.51
980  2000  10.60
981  2000  11.20
982  2000  12.32
983  2000  14.65, 'Vinnytsya':      year    vhi
980  2000  12.26
981  2000  11.28
982  2000  11.25
983  2000  11.38
984  2000  12.91
985  2000  14.20, 'Kharkiv':      year    vhi
981  2000  14.61
982  2000  11.33
983  2000   9.36
984  2000   9.45
985  2000   9.73
986  2000  11.45
987  2000  14.29, 'Kiev City':      year    vhi
933  1999  14.54
934  1999  13.54
935  1999  12.88
977  2000  14.89
978  2000  12.76
979  2000   7.81
980  2000   6.49
981  2000   6.58
982  2000   6.71
983  2000   7.56
984  2000   9.25
985  2000  10.94
986  2000  12.28, "Sevastopol'":      year    vhi
980  2000  13.14
981  2000   9.50
982  2000   8.14
983  2000   9.69
984  2000  11.20
985  2000  11.36
986  2000  12.77, 'Cherkasy':      year    vhi
979  2000  14.64
980  2000  11.82
981  2000  10.81
982  2000  10.68
983  2000  12.30
984  2000  14.24})


        o Аналогічно для помірних посух

In [13]:
def years_drought_m(dfs):
    years_drought = {}
    for df in dfs.items():
        id = df[0].split("vhi",2)[1]
        series_dr= df[1][(df[1].vhi > 15) & (df[1].vhi < 35) & (df[1].vhi != -1)]
        years_dr = series_dr['year'].unique()
        years_drought[f'{id}'] = years_dr 

    summary = []
    for k in years_drought.items():
        summary.extend(k[1])

    massive_drought = {i for i in summary if summary.count(i)>5}
    vhis = {}
    for k in years_drought.items():
        for i in massive_drought:
            if i in k[1]:
                id = k[0]
                province = get_province(id)
                df = dfs[f'vhi{id}']
                vhi = (df[(df.vhi > 15) & (df.vhi < 35) &(df.vhi != -1)][['year','vhi']])
                vhis[f'{province}'] = vhi
    return massive_drought, vhis

print (years_drought_m(dfs_mod))

({'2010', '2015', '1988', '1990', '1993', '2020', '1997', '2006', '1991', '1995', '2012', '2018', '1982', '2009', '1986', '2023', '2001', '2016', '2014', '2022', '1996', '2000', '1992', '1989', '2011', '2008', '2019', '2017', '2021', '2003', '2002', '1984', '1985', '2007', '1987', '1994', '2013', '1983', '1999'}, {'Chernivtsi':       year    vhi
10    1982  34.85
35    1982  32.89
36    1982  29.15
37    1982  26.97
38    1982  26.84
...    ...    ...
1965  2019  27.27
1966  2019  28.34
1967  2019  29.12
1968  2019  32.45
1969  2019  34.70

[217 rows x 2 columns], 'Sumy':       year    vhi
8     1982  34.15
9     1982  33.96
10    1982  33.78
45    1982  32.63
46    1982  29.46
...    ...    ...
2023  2020  30.81
2024  2020  33.86
2025  2020  32.76
2026  2020  34.78
2176  2023  34.22

[209 rows x 2 columns], 'Volyn':       year    vhi
6     1982  33.57
7     1982  33.23
8     1982  33.53
32    1982  32.19
33    1982  29.66
...    ...    ...
1075  2002  27.69
1076  2002  33.91
1748  201